In [ ]:
'''
sdf-2-sdf 论文相关 代码处理 @2018-3-29 21:40:44
数据页面：http://campar.in.tum.de/personal/slavcheva/3d-printed-dataset/synthetic/load_exr_depth.txt

opencv 代码：
cv::Mat load_exr_depth( std::string filename ) {
  // load the image
  cv::Mat depth_map = cv::imread( filename, -1 );
  cv::cvtColor( depth_map, depth_map, CV_RGB2GRAY );
  
  // convert to meters
  depth_map.convertTo( depth_map, CV_32FC1, 0.001 );
  
  return depth_map;
}

内参矩阵：
1, syn
http://campar.in.tum.de/personal/slavcheva/3d-printed-dataset/synthetic/intrinsics_synthetic.txt
570.3999633789062                   0   320.0
                0   570.3999633789062   240.0
                0                   0       1

2, kinect
http://campar.in.tum.de/personal/slavcheva/3d-printed-dataset/kinect/intrinsics_kinect.txt
538.925221           0  316.473843
         0  538.925221  243.262082
         0           0           1

3, industrial
http://campar.in.tum.de/personal/slavcheva/3d-printed-dataset/industrial/intrinsics_industrial.txt
2975.3781942               0    987.91809719392688
           0    2975.3781942    522.62119889236828
           0               0                     1
'''

In [146]:
%pylab
from glob import glob

import sys
sys.path.append(r'D:\opencv249\sources\modules\python\test')

import os
import cv2
import transformations as tf
'一些短别名:'
tf.e2m = tf.euler_matrix
tf.m2e = tf.euler_from_matrix
tf.q2m = tf.quaternion_matrix
tf.m2q = tf.quaternion_from_matrix
tf.e2q = tf.quaternion_from_euler
tf.q2e = tf.euler_from_quaternion
tf.m2r = tf.rotation_from_matrix
tf.r2m = tf.rotation_matrix
tf.qinv = tf.quaternion_inverse
tf.qmul = tf.quaternion_multiply

'angle <--> radian:'
a2r = np.pi / 180
r2a = 180 / np.pi

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [111]:
'''
用 *.exr 生成 ushort png; 针对仿真数据

'''
path = r"E:\sdf2sdf-data\Synthetic_Bunny_Circle\depth-raw"
path = r"E:\sdf2sdf-data\Synthetic_Bunny_Wave\depth-raw"
path = r"E:\sdf2sdf-data\Synthetic_Kenny_Circle\depth-raw"
path = r"E:\sdf2sdf-data\Synthetic_Kenny_Wave\depth-raw"
path = r"E:\sdf2sdf-data\Synthetic_Leopard_Circle\depth-raw"
path = r"E:\sdf2sdf-data\Synthetic_Leopard_Wave\depth-raw"
path = r"E:\sdf2sdf-data\Synthetic_Tank_Circle\depth-raw"
path = r"E:\sdf2sdf-data\Synthetic_Tank_Wave\depth-raw"
path = r"E:\sdf2sdf-data\Synthetic_Teddy_Circle\depth-raw"
path = r"E:\sdf2sdf-data\Synthetic_Teddy_Wave\depth-raw"

for fn in glob(path + r'\*.exr'):
    # fn = r"E:\sdf2sdf-data\Synthetic_Bunny_Circle\depth_000000.exr"
    #fpath_name, ext = os.path.splitext(fn)
    fpath, fname = os.path.split(fn)
    basename, ext = os.path.splitext(fname)
    dmap = cv2.imread(fn, -1)
    dmap = cv2.cvtColor(dmap, cv2.COLOR_RGB2GRAY)
    # dmap *= 1e-3 #mm2m
    dmap[dmap == np.inf] = 0
    dmap = dmap.astype(np.ushort) #.5 都被向下取整, 暂时这样
    dmap_mm = dmap * 1e-3
    cv2.imshow('dmap', dmap_mm)
    cv2.waitKey(20)
    
    '存文件: '
    new_path = os.path.join(fpath, 'depth')
    if not os.path.isdir(new_path):
        os.mkdir(new_path)
    new_fn = os.path.join(new_path, basename+'.png')
    cv2.imwrite(new_fn, dmap)
    pass

cv2.destroyAllWindows()

In [302]:
'''
把 kinect 采集的 markerboard_poses.txt 棋盘格外参 groundtruth 转成我之前用于 eval 的形式
@2018-4-7 00:19:12
'''

def reshape_poses(fn_src, fn_dst):
    rts = []
    poses = [] #N * 7 --> t3+q.wxyz

    with open(fn_src, 'r') as fin:
        for i, line in enumerate( fin.readlines() ):
            case = i % 5
            if case == 0:
                rt = []
                #continue
            else:
                rt.append( map(float, line.split()) )
                if case == 4:
                    rt = np.asarray(rt)
                    rts.append(rt); '暂无用'

                    rmat = rt.copy()
                    rmat[:3, 3] = 0
                    tvec = rt[:3, 3]

                    q = tf.m2q(rmat)
                    poses.append( hstack((tvec, q[3], q[:3])) ); '原本是 q.xyzw, 存成 t3+q4(wxyz)'
                    pass
                pass
            pass
        poses = np.asarray(poses)
        pass

    np.savetxt(fn_dst, poses, fmt='%.10f', delimiter=',')
    pass

'+++++++++++++++1, markerboard_poses'
path = r'E:\sdf2sdf-data\Kinect_Teddy_Turntable'
path = r'E:\sdf2sdf-data\Kinect_Tank_Turntable'
path = r'E:\sdf2sdf-data\Kinect_Kenny_Turntable'
path = r'E:\sdf2sdf-data\Kinect_Bunny_Turntable'

path = r'E:\sdf2sdf-data\Kinect_Bunny_Handheld'

fn_src = os.path.join(path, 'markerboard_poses.txt')
fn_dst = os.path.join(path, 'markerboard7n.csv')

'+++++++++++++++2, synthetic wave & circle poses'
# fn_src = r'E:\sdf2sdf-data\synthetic_wave_poses.txt'
# fn_dst = r'E:\sdf2sdf-data\synthetic_wave_poses.csv'

# fn_src = r'E:\sdf2sdf-data\synthetic_circle_poses.txt'
# fn_dst = r'E:\sdf2sdf-data\synthetic_circle_poses.csv'

print fn_dst
reshape_poses(fn_src, fn_dst)


E:\sdf2sdf-data\Kinect_Bunny_Handheld\markerboard7n.csv


In [295]:
'''
对 industrial 数据处理, 1080x2040->480x640
@2018-4-8 03:53:42
'''

# fn=r'C:\sdf2sdf-data\Industrial_Bunny\depth\depth_000000.bmp'
# dmap=cv2.imread(fn, cv2.IMREAD_UNCHANGED)
# dmap.shape
# where(dmap!=0)

path = r'C:\sdf2sdf-data\Industrial_Bunny'
path = r'C:\sdf2sdf-data\Industrial_Kenny'
path = r'C:\sdf2sdf-data\Industrial_Tank'
path = r'C:\sdf2sdf-data\Industrial_Leopard'
path = r'C:\sdf2sdf-data\Industrial_Teddy'

for fn in glob( os.path.join(path, r'depth-raw\*.bmp') ): #假设已预处理,手动把 bmp 移到 depth-raw 了
    fpath, fname = os.path.split(fn)
    basename, ext = os.path.splitext(fname)
    dmap = cv2.imread(fn, -1) #即 cv2.IMREAD_UNCHANGED
    dmmm = (dmap[:,:,0] + dmap[:,:,1] * 256 + dmap[:,:,2] * 65536) / 1e3 #转到 mm
    dm_vga = cv2.resize(dmmm, (640,480) ) #转到 vga, 此时仍是 float; resize 导致边界模糊错误, 原论文里不管
    dm_vga16u = (dm_vga + 0.5).astype( np.uint16 ) #这次小数位很精确, 所以 round nearest
    cv2.imshow('dm_vga', dm_vga/1e3) #绘制时要 mm->m
    cv2.waitKey(22)
    
    
    '存文件: 1, 全深度图'
    new_path = os.path.join(path, r'depth') #放到根目录的 depth/ 中
    if not os.path.isdir(new_path):
        os.mkdir(new_path)
    new_fn = os.path.join(new_path, basename+'.png')
    cv2.imwrite(new_fn, dm_vga16u)
    
    '2, -omask 后的深度图; 变量名不改, 可能覆盖了上面变量'
    fn_om = fn.replace('depth-raw\depth', 'omask\omask')
    fn_om = fn_om.replace('bmp', 'png')
    omsk = cv2.imread(fn_om, -1)
    dmmm[omsk==0] = 0
    dm_vga = cv2.resize(dmmm, (640,480) ) #转到 vga, 此时仍是 float; resize 导致边界模糊错误, 原论文里不管
    dm_vga16u = (dm_vga + 0.5).astype( np.uint16 ) #这次小数位很精确, 所以 round nearest

    new_path = os.path.join(path, r'depth-om') #放到根目录的 depth/ 中
    if not os.path.isdir(new_path):
        os.mkdir(new_path)
    new_fn = os.path.join(new_path, basename+'.png')
    cv2.imwrite(new_fn, dm_vga16u)

    pass

cv2.destroyAllWindows()

In [296]:
cv2.imshow('dm_vga', dm_vga/1e3) #绘制时要 mm->m


In [288]:
dmmm.copy?

In [286]:
cv2.imshow('asdf', omsk)

In [285]:
where(omsk!=0)
omsk[499, 942]

255

In [282]:
new_path

'C:\\sdf2sdf-data\\Industrial_Tank\\depth-om'

In [257]:
fn=r'C:\sdf2sdf-data\Industrial_Bunny\depth\depth_000000.png'
dmap=cv2.imread(fn, cv2.IMREAD_UNCHANGED)
print dmap.shape
print where(dmap!=0)
print dmap[39:44,184:188]

(480L, 640L)
(array([ 39,  39,  39, ..., 468, 468, 468], dtype=int64), array([184, 185, 186, ..., 240, 241, 242], dtype=int64))
[[238 239 637 637]
 [635 636 636 636]
 [634 635 635 636]
 [634 634 635 635]
 [633 634 634 635]]


In [256]:
fn=r'C:\sdf2sdf-data\Industrial_Bunny\depth-raw\depth_000000.bmp'
dmap=cv2.imread(fn, cv2.IMREAD_UNCHANGED)
dmap.shape
where(dmap!=0)


(array([  88,   88,   88, ..., 1053, 1053, 1053], dtype=int64),
 array([592, 592, 592, ..., 774, 774, 774], dtype=int64),
 array([0, 1, 2, ..., 0, 1, 2], dtype=int64))

In [207]:
108/48., 204/64.

(2.25, 3.1875)

In [235]:
dm2=(dmap[:,:,0]+dmap[:,:,1]*256+dmap[:,:,2]*65536)/1e3

In [232]:
dmap.transpose(2,0,1).shape

(3L, 1080L, 2040L)

In [236]:
dd=cv2.resize(dm2, (640,480))

In [238]:
dd*=1e-3
cv2.imshow('asdf', dd)

In [240]:
# print dmap[88:99,592:600]
print dmap[476:478, 903:906]
print dm2[476:478, 903:906]

print dmap[1053, 774]
# b, g, r=dmap[88,592]
# (r*2**16+g*2**8+b)/1e3

[[[ 51   0   9]
  [ 45   0   9]
  [253 255   8]]

 [[227 255   8]
  [220 255   8]
  [149 255   8]]]
[[ 589.875  589.869  589.821]
 [ 589.795  589.788  589.717]]
[73 36  8]


In [185]:
cv2.*UNCHA*?

In [188]:
print cv2.IMREAD_UNCHANGED
print cv2.CV_LOAD_IMAGE_UNCHANGED

-1
-1


In [112]:
fn = r'E:\sdf2sdf-data\Synthetic_Bunny_Circle\depth\depth_000000.png'
dmap = cv2.imread(fn, -1)
int(dmap[124,307])

522

In [33]:
print dmap[124:133, 307:333]

[[ 522.5  524.   525.5  526.5  528.5    inf    inf    inf    inf    inf
     inf    inf    inf    inf    inf    inf    inf    inf    inf    inf
     inf    inf    inf    inf    inf    inf]
 [ 522.   523.5  524.5  526.   527.5  529.5    inf    inf    inf    inf
     inf    inf    inf    inf    inf    inf    inf    inf    inf    inf
     inf    inf    inf    inf    inf    inf]
 [ 522.   523.5  524.5  526.   527.5  529.   531.5    inf    inf    inf
     inf    inf    inf    inf    inf    inf    inf    inf    inf    inf
     inf    inf    inf    inf    inf    inf]
 [ 522.5  523.5  525.   526.   527.5  528.5  530.5    inf    inf    inf
     inf    inf    inf    inf    inf    inf    inf    inf    inf    inf
     inf    inf    inf    inf    inf    inf]
 [ 522.5  524.   525.   526.5  527.5  528.5  530.   532.   534.     inf
     inf    inf    inf    inf    inf    inf    inf    inf    inf    inf
     inf    inf    inf    inf    inf    inf]
 [ 523.   524.   525.5  526.5  527.5  528.5  529.5  531

In [17]:
cv2.cvtColor(dmap, cv2.COLOR_RGB2GRAY)

array([[ inf,  inf,  inf, ...,  inf,  inf,  inf],
       [ inf,  inf,  inf, ...,  inf,  inf,  inf],
       [ inf,  inf,  inf, ...,  inf,  inf,  inf],
       ..., 
       [ inf,  inf,  inf, ...,  inf,  inf,  inf],
       [ inf,  inf,  inf, ...,  inf,  inf,  inf],
       [ inf,  inf,  inf, ...,  inf,  inf,  inf]], dtype=float32)